In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


from keras.layers import Input, Dense, Flatten,Activation
from keras.layers import Conv1D, MaxPooling1D,Dropout,LSTM
from keras.layers import Embedding

from numpy import zeros
from numpy import asarray

from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.layers import Dense
from keras.layers import Flatten

from keras.layers.embeddings import Embedding

from gensim.models import FastText

In [13]:
sentences = open('../data/dataset_for_char_rnn.txt').readlines()

In [14]:
for i in range(len(sentences)):
    sentences[i] = sentences[i].strip().split(' ')
sentences[:3]

[['амаравкэваратэн', 'таа’койӈын'],
 ['йъйыӄык', 'ныӄэԓпэратӄэн', 'вытэчгытрыӄэргыԓьын', 'йыӈэттэт'],
 ['мыкыӈ', 'нывытрэтӄин', 'чеԓгатвытрыԓьо', 'ынӄорыым', 'вытэчгытрыԓьо']]

In [15]:
fasttext_model = FastText(size=10, window=4, min_count=1)
fasttext_model.build_vocab(sentences)
fasttext_model.train(sentences, total_examples=len(sentences), epochs=10)


In [16]:
fasttext_model['йъйыӄык']

ipykernel_launcher:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


array([ 0.89108765,  0.23914945,  0.7755417 ,  0.5666302 , -2.3441646 ,
       -0.37617072, -1.2107517 ,  0.36574322,  1.1333429 ,  0.575741  ],
      dtype=float32)

In [24]:
result_dict_common = dict()

training_file = '../data/train.tsv'

for i, line in enumerate(open(training_file).readlines()):
    row = line.strip().split('\t')
    tokens = row[0]
    for id_tok, tok in enumerate(tokens.split(' ')):
        if id_tok in result_dict_common:
            result_dict_common[id_tok].append(tok)
        else:
            result_dict_common[id_tok] = list()
            result_dict_common[id_tok].append(tok)

In [17]:
df = pd.read_csv('../data/df_for_train_classifire.csv', index_col=0)

In [18]:
df.head(2)

,type,text,label
0,2,амаравкэваратэн,таа’койӈын
1,2,йъйыӄык,ныӄэԓпэратӄэн


In [19]:
df.type.value_counts()[:5]

2    26473
3    20768
4    15301
5    10986
6     7741
Name: type, dtype: int64

In [29]:
result_dict = dict()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.text.tolist())

vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index
EMBEDDING_DIM = 10

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, i in word_index.items():
    try:
        embedding_vector = fasttext_model[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)


all_types = pd.unique(df.type).tolist()

for id_t, t in enumerate(all_types):
    print('Current type', id_t, 'All type', len(all_types))
    tmp_df = df[df.type == t]
    tmp_df = tmp_df.drop(['type'], axis=1)

    result_dict[t] = dict()
    result_dict[t]['le'] = LabelEncoder()
    tmp_df['label'] = result_dict[t]['le'].fit_transform(tmp_df['label'])

    labels_len = len(pd.unique(tmp_df['label']).tolist())

    encoded_docs = tokenizer.texts_to_sequences(tmp_df.text.tolist())
    
    padded_docs = pad_sequences(encoded_docs, maxlen=t-1, padding='post')

    labels = to_categorical(np.asarray(tmp_df['label'].tolist()))


    # result_dict[t]['model'] = Sequential()
    # result_dict[t]['model'].add(Embedding(result_dict[t]['vocab_size'], 10, input_length=t-1))
    # result_dict[t]['model'].add(Activation('relu'))
    # result_dict[t]['model'].add(Dropout(0.1))
    # result_dict[t]['model'].add(Flatten())
    # result_dict[t]['model'].add(Dense(30))
    # result_dict[t]['model'].add(Activation('relu'))
    # result_dict[t]['model'].add(Dense(labels_len))
    # result_dict[t]['model'].add(Activation('softmax'))
    # result_dict[t]['model'].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # result_dict[t]['model'].fit(padded_docs, labels, epochs=5, verbose=1)

    if id_t > 15:
        epoh_n = 10
    else:
        epoh_n = 50

    result_dict[t]['model'] = Sequential()
    result_dict[t]['model'].add(Embedding(vocab_size, EMBEDDING_DIM, input_length=t-1, weights=[embedding_matrix], trainable=False))
    result_dict[t]['model'].add(Dropout(0.2))
    result_dict[t]['model'].add(LSTM(64))
    result_dict[t]['model'].add(Dropout(0.2))
    result_dict[t]['model'].add(Dense(labels_len, activation='softmax'))
    result_dict[t]['model'].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    result_dict[t]['model'].fit(padded_docs, labels, epochs=epoh_n, verbose=1)

- 0s 518us/step - loss: 4.8360 - acc: 0.0145
Epoch 4/10
138/138 [==============================] - 0s 523us/step - loss: 4.8247 - acc: 0.0217
Epoch 5/10
138/138 [==============================] - 0s 497us/step - loss: 4.8250 - acc: 0.0217
Epoch 6/10
138/138 [==============================] - 0s 465us/step - loss: 4.8144 - acc: 0.0145
Epoch 7/10
138/138 [==============================] - 0s 492us/step - loss: 4.8214 - acc: 0.0145
Epoch 8/10
138/138 [==============================] - 0s 527us/step - loss: 4.7954 - acc: 0.0145
Epoch 9/10
138/138 [==============================] - 0s 522us/step - loss: 4.8099 - acc: 0.0217
Epoch 10/10
138/138 [==============================] - 0s 501us/step - loss: 4.7804 - acc: 0.0217
Current type 17 All type 37
Epoch 1/10
99/99 [==============================] - 7s 69ms/step - loss: 4.5397 - acc: 0.0101
Epoch 2/10
99/99 [==============================] - 0s 630us/step - loss: 4.5221 - acc: 0.0000e+00
Epoch 3/10
99/99 [==============================] - 0s

In [30]:
f = open('output.tsv', 'w')

n_tokens = 0
hits = 0

for line in tqdm(open('../data/dev.tsv').readlines()):
    # Split into two columns
    row = line.strip().split('\t')
    # Our tokens are in column one, split by space
    tokens = row[0].split(' ')
    # The test tokens are the beginning of sentence symbol + the list of tokens
    tst_tokens = ['#'] + tokens
    # Increment the number of tokens by the length of the list containing the tokens
    n_tokens += len(tokens)

    # This is our output
    output = []

    # For each of the tokens in the "tst_tokens" list (e.g. the list + the beginning of sentence symbol)
    for i in range(len(tst_tokens)-1):
        first = tst_tokens[i]  # First token in bigram
        second = tst_tokens[i+1]  # Second token in bigram

        if i == 0:
            predicted_second = pd.Series(result_dict_common[0]).value_counts()[:1].index[0]
        else:
            exits_text = ' '.join(tst_tokens[1:i+1])
            predicted_second = result_dict[i+1]['model'].predict(pad_sequences(tokenizer.texts_to_sequences([exits_text]), maxlen=i, padding='post'))
            predicted_second = np.argmax(predicted_second)
            predicted_second = result_dict[i+1]['le'].inverse_transform([predicted_second])
            predicted_second = predicted_second[0]
        

        if predicted_second == second:
            # We add this whole token to the output
            # e.g. a single click on a prediction
            output.append(predicted_second)
            # Increment the number of hits by 1
            hits += 1
        else:
            # Otherwise we add each individual character to the output
            # e.g. writing out each of the individual clicks
            output += [c for c in second]

        output.append('_')

    
    print('%s\t%s' % (row[0], ' '.join(output)), file=f)

f.close()

100%|██████████| 1000/1000 [01:22<00:00, 12.09it/s]


In [31]:
hits

35

In [32]:
!python3.7 ../evaluate.py ../data/dev.tsv ./output.tsv

Characters: 37897
Tokens: 8788
Clicks: 37754
Clicks/Token: 4.2960855712335
Clicks/Character: 0.9962266142438716
